In [ ]:
from utils.Pipeline.grammar_score import AdvancedGrammarScorer  # 문법 검사 클래스
from utils.Pipeline.consistency import measure_sc  # 의미 일관성 검사 함수
from utils.Pipeline.grammar_corrector import (
    GrammarCorrector,
)  # Grammarformer를 사용한 문장 보완 클래스


class SentenceEvaluationPipeline:
    def __init__(self, grammar_threshold: float = 0.9, coherence_threshold: int = 1):
        self.grammar_scorer = AdvancedGrammarScorer()  # 문법 점수 측정
        self.coherence_checker = measure_sc  # 의미 일관성 점검 함수
        self.grammar_threshold = grammar_threshold  # 문법 점수 기준
        self.coherence_threshold = coherence_threshold  # 일관성 기준
        self.grammar_corrector = GrammarCorrector()  # GrammarCorrector 초기화

    def evaluate_and_correct(self, sentence: str):
        # 1. 문법 점수 계산
        grammar_score, grammar_details = self.grammar_scorer.score_sentence(sentence)
        print(f"Initial Grammar Score: {grammar_score}")

        # 2. 문법 점수 기준 충족 여부 확인
        if grammar_score < self.grammar_threshold:
            print(
                "Grammar score below threshold, attempting correction with Grammarformer."
            )
            sentence = self.grammar_corrector.correct_sentence(
                sentence
            )  # GrammarCorrector로 교정
            grammar_score, _ = self.grammar_scorer.score_sentence(
                sentence
            )  # 수정 후 다시 점검
            print(f"Updated Grammar Score: {grammar_score}")

        # 3. 의미 일관성 점검
        coherence_score = self.coherence_checker(sentence)
        print(f"Initial Coherence Score: {coherence_score}")

        # 4. 의미 일관성 기준 충족 여부 확인
        if coherence_score < self.coherence_threshold:
            print("Coherence score below threshold, attempting further correction.")
            sentence = self.grammar_corrector.correct_sentence(
                sentence
            )  # GrammarCorrector로 보완
            coherence_score = self.coherence_checker(sentence)  # 수정 후 다시 점검
            print(f"Updated Coherence Score: {coherence_score}")

        # 최종 결과 반환
        return {
            "final_sentence": sentence,
            "grammar_score": grammar_score,
            "coherence_score": coherence_score,
        }


# 사용 예시
if __name__ == "__main__":
    pipeline = SentenceEvaluationPipeline()
    input_sentence = "He have one cars."
    result = pipeline.evaluate_and_correct(input_sentence)

    print("Final Sentence:", result["final_sentence"])
    print("Final Grammar Score:", result["grammar_score"])
    print("Final Coherence Score:", result["coherence_score"])

Initial Grammar Score: 0.74
Grammar score below threshold, attempting correction with Grammarformer.
Updated Grammar Score: 0.99
Initial Coherence Score: 1
Final Sentence: He has one car.
Final Grammar Score: 0.99
Final Coherence Score: 1


In [1]:
import pandas as pd
from Pipeline.grammar_score import AdvancedGrammarScorer  # 문법 검사 클래스
from Pipeline.consistency import measure_sc  # 의미 일관성 검사 함수
from Pipeline.grammar_corrector import GrammarCorrector  # Grammarformer를 사용한 문장 보완 클래스

class SentenceEvaluationPipeline:
    def __init__(self, grammar_threshold: float = 0.7, coherence_threshold: int = 1):
        self.grammar_scorer = AdvancedGrammarScorer()  # 문법 점수 측정
        self.coherence_checker = measure_sc  # 의미 일관성 점검 함수
        self.grammar_threshold = grammar_threshold  # 문법 점수 기준
        self.coherence_threshold = coherence_threshold  # 일관성 기준
        self.grammar_corrector = GrammarCorrector()  # GrammarCorrector 초기화

    def evaluate_and_correct(self, sentence: str):
        # 1. 문법 점수 계산
        grammar_score, grammar_details = self.grammar_scorer.score_sentence(sentence)
        print(f"Initial Grammar Score: {grammar_score}")
        
        # 2. 문법 점수 기준 충족 여부 확인
        if grammar_score < self.grammar_threshold:
            print("Grammar score below threshold, attempting correction with Grammarformer.")
            sentence = self.grammar_corrector.correct_sentence(sentence)  # GrammarCorrector로 교정
            grammar_score, _ = self.grammar_scorer.score_sentence(sentence)  # 수정 후 다시 점검
            print(f"Updated Grammar Score: {grammar_score}")
        
        # 3. 의미 일관성 점검
        coherence_score = self.coherence_checker(sentence)
        print(f"Initial Coherence Score: {coherence_score}")
        
        # 4. 의미 일관성 기준 충족 여부 확인
        if coherence_score < self.coherence_threshold:
            print("Coherence score below threshold, attempting further correction.")
            sentence = self.grammar_corrector.correct_sentence(sentence)  # GrammarCorrector로 보완
            coherence_score = self.coherence_checker(sentence)  # 수정 후 다시 점검
            print(f"Updated Coherence Score: {coherence_score}")
        
        # 최종 결과 반환
        return {
            "final_sentence": sentence,
            "grammar_score": grammar_score,
            "coherence_score": coherence_score
        }

# 파일에서 데이터 로드 및 처리
def process_all_sentences(file_path: str):
    # CSV 파일 로드
    df = pd.read_csv(file_path)
    pipeline = SentenceEvaluationPipeline()

    # 모든 문장에 대해 파이프라인 실행
    results = []
    for _, row in df.iterrows():
        sentence = row['문장']
        word = row['포함된 단어']
        
        # 파이프라인 평가 및 수정
        print(f"검사문장: '{sentence}' 학습단어: '{word}'")
        result = pipeline.evaluate_and_correct(sentence)
        result["word"] = word  # 단어 정보 추가
        results.append(result)

    # 결과 출력
    for res in results:
        print("\nProcessed Result:")
        print("Word:", res["word"])
        print("Final Sentence:", res["final_sentence"])
        print("Final Grammar Score:", res["grammar_score"])
        print("Final Coherence Score:", res["coherence_score"])

# 파일 경로 설정 및 실행
file_path = 'C:/Users/user/Desktop/eng-word/utils/Grammar_Score/grammar_sentences.csv'
process_all_sentences(file_path)


c:\Users\user\Desktop\Chunjae_edu\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


검사문장: 'I have an aunt who lives nearby.' 학습단어: 'aunt'
Initial Grammar Score: 1.01
Initial Coherence Score: 1
검사문장: 'I’m going to the concert hall.' 학습단어: 'be going to'
Initial Grammar Score: 0.98
Initial Coherence Score: 1
검사문장: 'I scanned the boy.' 학습단어: 'boy'
Initial Grammar Score: 0.98
Initial Coherence Score: 1
검사문장: 'I exercise every morning to stay healthy.' 학습단어: 'exercise'
Initial Grammar Score: 0.99
Initial Coherence Score: 1
검사문장: 'I know The girl is my sister and this is my doll.' 학습단어: 'girl'
Initial Grammar Score: 0.9
Initial Coherence Score: 0
Coherence score below threshold, attempting further correction.


KeyboardInterrupt: 

In [1]:
df = pd.read_csv("C:/Users/user/Desktop/eng-word/utils/Grammar_Score/grammar_sentences.csv")
df


NameError: name 'pd' is not defined

In [2]:
!pip install pandas